Agentic Project Agnostic Management Advisor - Project Planning Module

Authors			    :	Kyaw Kyaw Oo<br>
Emails              :   ook1@mcmaster.ca<br>
Student No          :   400551761<br>
Group No			:   Sixth Smart Sense Generation<br>
Started Date        :   Feb 6, 2025<br>
Version             :   1<br>
Description         :   Skeleton<br>
Version             :   2<br>
Description         :   Update the yaml files align to the problem statements<br>
Version             :   3<br>
Description         :   Fine tune the yaml files and redefine the agents and tasks<br>
Version             :   4<br>
Description         :   Creat Crew and Fine tune the output format <br>
Version             :   5<br>
Description         :   Validate the output<br>
Version             :   6<br>
Description         :   Code Clean, Refactor and Check the Output again<br>
Released Date    	: 	Apr 5, 2025 <br>
Lecturer            :   Dr Hamidreza Mahyar<br>
TA                  :   Ali Shiraee Kasmaee <br>

CopyRight@2024 by Sixth Smart Sense Generation<br>

In [ ]:

# %pip install crewai python-dotenv --q
# %pip install --upgrade crewai --q


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.113 requires chromadb<0.5.0,>=0.4.24, but you have chromadb 0.6.3 which is incompatible.
langchain-community 0.0.38 requires tenacity<9.0.0,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
langchain-core 0.1.53 requires tenacity<9.0.0,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
crewai-tools 0.1.6 requires chromadb<0.5.0,>=0.4.22, but you have chromadb 0.6.3 which is incompatible.
langchain-experimental 0.3.4 requires langchain-community<0.4.0,>=0.3.0, but you have langchain-community 0.0.38 which is incompatible.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have lan

In [2]:
# Load Basic Libraries  
import os 
import yaml


In [3]:

from IPython.display import display, Markdown
from typing import List

from dotenv import load_dotenv

In [4]:

from pydantic import BaseModel, Field

from crewai import Agent, Task, Crew, LLM

In [5]:
# Set Up 
load_dotenv()

True

In [6]:

# Reading configurations
# Define file paths for YAML configurations
# input from front end
# read_config
files = {
    'agents': 'config/project_planning_agents.yaml',
    'tasks': 'config/project_planning_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

#test config_data
# print(agents_config)
# print(agents_config['project_planning_agent']

# Setup LLM 
# load_llm
ollama_llm = LLM(
            model="ollama/llama3.2:latest",
            api_base="http://localhost:11434"
            )

# define_project
class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

# create_agents
# Creating Agents and Tasks
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm =  ollama_llm,
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm =  ollama_llm,
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm =  ollama_llm,
)

# create_tasks
# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# create_crew
# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

# Crew's Input

In [7]:

project = 'Sale Promotion'
industry = 'Retails'
project_objectives = 'Create a campaign for a product that is not primarily aimed at generating revenue or profit.'
team_members = """
- John Doe (Marketing Strategist)
- Amy Johnson (Creative Director)
- Bob Smith (Social Media Manager)
- Alice Hung (Data Analyst)
- Tom Brown (Public Relations Specialist)
- Ethan Wilson (Event Coordinator)
"""
project_requirements = """
- Conduct thorough market research to understand the target audience, competitors, and current market trends
- Develop a unique and compelling creative concept that resonates with the target audience and aligns with the brand's values
- Design a comprehensive marketing strategy that leverages multiple channels, including social media, email, in-store promotions, and events
- Produce engaging content, including graphics, videos, and written materials, to support the campaign across all platforms
- Implement initiatives to engage customers, such as contests, giveaways, and interactive experiences, to increase participation and interest.
- Establish key performance indicators (KPIs) and track the campaign's performance, providing regular reports and insights for optimization.
- Develop and manage the campaign budget, ensuring cost-effectiveness and efficient allocation of resources.
- Identify and collaborate with potential partners, influencers, or other brands to amplify the campaign's reach and impact.
- Gather customer feedback and adapt the campaign as needed to improve effectiveness and achieve objectives.
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}

**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Sale Promotion

**Project Objectives:** Create a campaign for a product that is not primarily aimed at generating revenue or profit.

**Industry:** Retails

**Team Members:**

- John Doe (Marketing Strategist)
- Amy Johnson (Creative Director)
- Bob Smith (Social Media Manager)
- Alice Hung (Data Analyst)
- Tom Brown (Public Relations Specialist)
- Ethan Wilson (Event Coordinator)


**Project Requirements:**

- Conduct thorough market research to understand the target audience, competitors, and current market trends
- Develop a unique and compelling creative concept that resonates with the target audience and aligns with the brand's values
- Design a comprehensive marketing strategy that leverages multiple channels, including social media, email, in-store promotions, and events
- Produce engaging content, including graphics, videos, and written materials, to support the campaign across all platforms
- Implement initiatives to engage customers, such as contests, giveaways, and interactive experiences, to increase participation and interest.
- Establish key performance indicators (KPIs) and track the campaign's performance, providing regular reports and insights for optimization.
- Develop and manage the campaign budget, ensuring cost-effectiveness and efficient allocation of resources.
- Identify and collaborate with potential partners, influencers, or other brands to amplify the campaign's reach and impact.
- Gather customer feedback and adapt the campaign as needed to improve effectiveness and achieve objectives.



### Kicking off the crew

In [8]:
# Inputs from Web App
# inputs as Dictionary or Json Format

In [10]:
# The given Python dictionary
# perform_crew
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

 
[2025-03-26 10:30:49][🚀 CREW 'CREW' STARTED, BACA100D-E63F-4B13-B5F6-2CF4A92F6102]: 2025-03-26 10:30:49.416735
 
[2025-03-26 10:30:49][📋 TASK STARTED: CAREFULLY ANALYZE THE PROJECT_REQUIREMENTS FOR THE SALE PROMOTION PROJECT AND BREAK THEM DOWN INTO INDIVIDUAL TASKS. DEFINE EACH TASK'S SCOPE IN DETAIL, SET ACHIEVABLE TIMELINES, AND ENSURE THAT ALL DEPENDENCIES ARE ACCOUNTED FOR:

- CONDUCT THOROUGH MARKET RESEARCH TO UNDERSTAND THE TARGET AUDIENCE, COMPETITORS, AND CURRENT MARKET TRENDS
- DEVELOP A UNIQUE AND COMPELLING CREATIVE CONCEPT THAT RESONATES WITH THE TARGET AUDIENCE AND ALIGNS WITH THE BRAND'S VALUES
- DESIGN A COMPREHENSIVE MARKETING STRATEGY THAT LEVERAGES MULTIPLE CHANNELS, INCLUDING SOCIAL MEDIA, EMAIL, IN-STORE PROMOTIONS, AND EVENTS
- PRODUCE ENGAGING CONTENT, INCLUDING GRAPHICS, VIDEOS, AND WRITTEN MATERIALS, TO SUPPORT THE CAMPAIGN ACROSS ALL PLATFORMS
- IMPLEMENT INITIATIVES TO ENGAGE CUSTOMERS, SUCH AS CONTESTS, GIVEAWAYS, AND INTERACTIVE EXPERIENCES, TO INCREASE 

### Usage Metrics and Costs

In [11]:
# Output to Web App
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0015


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,9771,6377,0,3394,4


### Result

In [12]:
print(result.raw)
# output to front_end
# Output to Web App

** 

I've completed the Sale Promotion project with a total effort estimation of 600 hours, spread across 20 weeks. The project requires high-effort tasks such as creative concept development (2 weeks), marketing strategy design (2 weeks), content production (2 weeks), customer engagement initiatives (2 weeks), and finalizing and launching the campaign (2 weeks). 

**Gantt Chart:**

| Task | Start Date | End Date | Duration | Dependencies |
| --- | --- | --- | --- | --- |
| 1. Market Research | Week 1 | Week 2 | 2 weeks | - |
| 2. Creative Concept Development | Week 3 | Week 4 | 2 weeks | Task 1 completed |
| 3. Marketing Strategy Design | Week 5 | Week 6 | 2 weeks | Tasks 1 & 2 partially completed |
| 4. Content Production | Week 7 | Week 8 | 2 weeks | Task 3 partially completed |
| 5. Customer Engagement Initiatives | Week 9 | Week 10 | 2 weeks | Tasks 3-4 partially completed |
| 6. KPI Establishment and Tracking | Week 11 | Week 12 | 2 weeks | Tasks 3-5 completed, Task 7 partially c

In [ ]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)
# output to front_end
# Output to Web Appç

,task_name,estimated_time_hours,required_resources
0,Market Research,2.000000,[]
1,Creative Concept Development,2.000000,[]
2,Marketing Strategy Design,2.000000,[]
3,Content Production,2.000000,[]
4,Customer Engagement Initiatives,2.000000,[]
5,KPI Establishment and Tracking,2.000000,[]
6,Budget Management,2.000000,[]
7,Partner Identification and Collaboration,2.000000,[]
8,Customer Feedback Collection and Campaign Adaptation,2.000000,[]
9,Finalize and Launch Campaign,2.000000,[]


### Inspecting Milestones

In [14]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

# output to front_end
# Output to Web App

,milestone_name,tasks
0,Sale Promotion project completed,"['Market Research completed', 'Creative Concept Development completed', 'Marketing Strategy Design partially completed', 'Content Production partially completed', 'Customer Engagement Initiatives partially completed']"
